In [1]:
# TODOS:

# Make model object (with empty functions and instance variables) -- Start the template
# - sample() - from GMM -- (already exists code) -- Katherine x
# - generate_palette() -- Anna
# - output() - output in json (labelling. check data/gallery for format but up to change if u think it's ugly) ** Katherine
# ** Intelligently make stepping wheel (sometimes monochromatic) -- Anna [focus on 2 colors?]

# REFERENCES: https://www.slynyrd.com/blog/2018/1/10/pixelblog-1-color-palette
# HEURISTICS FOR HSL: http://hslpicker.com/

In [2]:
import sys
!{sys.executable} -m pip install colr

In [3]:
from colormath.color_objects import LabColor, XYZColor, sRGBColor, HSLColor, AdobeRGBColor
from colormath.color_conversions import convert_color
from colr import Colr as C
from copy import copy, deepcopy

import random as r
import numpy as np
import itertools

from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse

class color_library:
    """ Class that represents a color library. The color palette has access to this. """

    def rgb_to_hsl(self, a, b, c):
        rgb = sRGBColor(a, b, c, is_upscaled=True)
        hsl = convert_color(rgb, HSLColor)
        return hsl.get_value_tuple()

    def hsl_to_rgb(self, a, b, c):
        hsl = HSLColor(a, b, c)
        rgb = convert_color(hsl, sRGBColor).get_value_tuple()

        return [rgb[0] * 255, rgb[1]*255, rgb[2]*255]
    
    def arr_to_int(self, arr):
        for r in range(len(arr)):
            arr[r] = int(arr[r])
        return arr

    # rgb inputs
    def print_combo(self, fg, bg):
        for i in range(0,3):
            if fg[i] > 255:
                fg[i] = 255
            if bg[i] > 255:
                bg[i]= 255
        print(C().b_rgb(bg[0], bg[1], bg[2]) .rgb(fg[0], fg[1], fg[2], 'Lorem ipsum.'))

    def bound(self, min_val, max_val, val):
        new_val = val
        if (val > max_val):
             new_val = max_val
        elif (val < min_val):
            new_val = min_val
        return new_val
    
    def rgb_to_hex(self, r,g,b):
        return '#%02x%02x%02x' % (int(r), int(g), int(b))
    
    def color_descriptor(self, hue, saturation, lightness):
#         if saturation < 10:
#             return "grey"
        if lightness == 0:
            return "black"
        elif lightness == 100:
            return "white"
        elif hue <= 10 or hue >= 350:
            return "red"
        elif hue < 40:
            return "orange"
        elif hue < 60:
            return "yellow"
        elif hue < 160:
            return "green"
        elif hue < 250:
            return "blue"
        elif hue < 290:
            return "purple"
        elif hue < 350:
            return "pink"


In [6]:
class palette_generator:
    """ Class that represents the palette generator model """

    def __init__(self):
        self.color_library = color_library()
        self.gmm = GaussianMixture(n_components=2)
        
    # Helper method for testing
    # input: array of hsl samples directly taken from the gmm and converts them
    # into rgb and prints them!
    def print_samples(self, samples):
        print("----printing samples----")
        for color in samples:
            rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
            self.color_library.print_combo(rgb,rgb)
        
    
    # Given a list of inputs [colors] that are in RGB form [[r, g, b], [r, g, b]]
    # Outputs colors in HSL: [ [h,s,l], [h,s,l]]
    def sample_gmm(self, samples, if_print_samples, num_samples):
        hsl_likes = []
        
        print("given samples:")
        for color in samples:
            if if_print_samples: 
                self.color_library.print_combo(color, color)
            hsl_likes.append(self.color_library.rgb_to_hsl(color[0], color[1], color[2]))
        
        hsl_likes = np.reshape(hsl_likes, (-1, 3))
        self.gmm.fit(hsl_likes)
        return self.gmm.sample(num_samples)[0]
    
    def saturation_clip(self, value):
        value = abs(value)
        direction = (-1)**int(value)
        return np.mod((1 + direction*np.mod(value, 1)), 1)
        
    # return palettes in form [ [color, color...color], [color, color, color]]., where each color is
    # an array of hsl colors [h,s,l]
    # [[[h,s,l], [h,s,l]...], [[h,s,l], [h,s,l] ...]].
    # ANNA: TODO 
    def generate_palettes(self, samples, num_palettes, if_print_inputs, num_steps):
        samples = self.sample_gmm(samples, if_print_inputs, num_palettes)
#         print("gmm samples:")
#         self.print_samples(samples) # to print gmm samples
        
        palettes = []
        for color in samples: # make a palette for each sample from the gmm
            palettes.append(self.stepping_wheel(color, num_steps))
#             print(" -- ")
        return palettes

    # for anna to play around with!
    def stepping_wheel(self, color, steps):
#         print("given color: " + str(color))
        rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
        # self.color_library.print_combo(rgb, rgb)

        # 0 = hue, 1 = saturation, 2 = lightness.
        center_i = round(steps/2 + 0.1)-1
            
        # hue: linear increase
        palette_h = np.zeros(steps)
        hue_shift = 20
        for i in range(0, steps):
            value = color[0] + (i - center_i) * hue_shift
            palette_h[i] = np.mod(value, 360)
        
        # saturation: porportional decrease
        palette_s = np.zeros(steps)         
        saturation_shift = 0.2
        palette_s[center_i] = np.clip(color[1], 0, 1)
        sign = 1
        for i in range(center_i - 1, -1, -1): # left
            last = palette_s[i + 1] # saturation of last element
            prop = 1.0/(1 + np.exp(abs(i - center_i)/5.0)) 
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        sign = 1
        for i in range(center_i + 1, steps, 1): # right   
            last = palette_s[i - 1] # saturation of last element
            prop = 1.0/(1 + np.exp(abs(i - center_i)/5.0)) 
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        palette_s = np.clip(palette_s, 0.01, 0.99)
        
        # brightness: a*log(x)+1, a in [0.3, 1] (increasing)
        palette_b = np.zeros(steps)
        palette_b[center_i] = color[2] # given middle color
        delta_x = 0.05
        # setting a
        if (color[2] > 0.45 and color[2] < 0.55):
            a = 1.6245 # 2**0.7
        else:
            a = abs(0.1/(color[1]-0.5))**0.7  
#         print("a: " + str(a))
        for i in range(center_i - 1, -1, -1): # left
            last = palette_b[i + 1] # brightness of last element
            palette_b[i] = last - delta_x * (a/last)**0.5 # derivative of a*log(x)+1          
        for i in range(center_i + 1, steps, 1): # right   
            last = palette_b[i - 1] # brightness of last element
            palette_b[i] = last + delta_x * (a/last)**0.5 # derivative of a*log(x)+1
        palette_b = np.clip(palette_b, 0.01, 0.95)
        
        # combining h, s, b
        palette = []
        print("palette:")
        for i in range(0, steps):
            c = [palette_h[i], palette_s[i], palette_b[i]]
#             print(c)
            palette.append(c) 
            rgb = self.color_library.hsl_to_rgb(c[0], c[1], c[2])
            self.color_library.print_combo(rgb, rgb)

        return palette
    
    
    # helper function that translates generated palettes into a json file. (outputs of generate_palettes)
    # inputs: palettes in form [ [ color, .., color], [color, ..., color] ]  where each color is hsl
    # output: a json file in format:
    # [{id: color1{label, hex, rgb}, {color2:label, hex, rgb}]
    def output_to_json(self, palettes):
        outputs = {}
        
        for palette in palettes:
            c_counter = 1
            color_id = []
            p = {}
            for color in palette:
                c = {}
                rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
                c["rgb"] = self.color_library.arr_to_int(rgb)
                c["hex"] = self.color_library.rgb_to_hex(rgb[0], rgb[1], rgb[2])
                c["label"] = [self.color_library.color_descriptor(color[0], color[1], color[2])]
                color_string = "color" + str(c_counter)
                p[color_string] = c
                c_counter +=1
                color_id.append(c["hex"])
            
            outputs[hash(tuple(color_id))] = p # note: we want to order the palettes from light to dark before hashing!
        
        return outputs
    
        

In [7]:
# a list of colors we like in rgb, pastels
import json
pastel = [ [255, 228, 171], [255, 171, 209], [144, 240, 155], [245, 118, 130], [250, 178, 162], [145, 255, 187], [203, 240, 168]]

# a list of earth tones
earth = [ [192, 87, 70], [240, 207, 101], [73, 67, 49], [89, 152, 197], [222, 185, 134], [208, 205, 148], [247, 208, 138]]

wack = [ [246,71,64], [248,221,164], [191, 219, 247], [60, 187, 177], [87, 226, 229], [241, 113, 5], [106, 16, 242]]

dark = [ [140, 6, 4], [25, 3, 69] ,[3, 32, 43], [25, 3, 69], [41, 30, 35], [20, 18, 19], [2, 38, 4] ]

# [25, 3, 69], [41, 30, 35], [20, 18, 19], [2, 38, 4]
generator = palette_generator()
print("---")

# note: dark fails on 5
palettes = generator.generate_palettes(pastel, 10, True, 8)
# print("palettes", palettes)


# data =  generator.output_to_json(palettes)

# print(data)

# dumps data into json after
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(data, f, ensure_ascii=False, indent=4)

---
given samples:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
palette:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
palette:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
palette:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
palette:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
palette:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
palette:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
palette:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
palette:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.